Based off of work in vocab.ipynb, this program extracts tobacco and drug related bigrams from each text in a given time period. The later code includes work done to categorize co-occurring words to guide the coloring of edges in period networks

In [ ]:
from collections import Counter
import os
import pandas as pd
from gensim.models import Phrases, phrases

def getTexts(folder,searchList):
    fileToText = {}
    underscores = {}
    for root,dirs,files in os.walk(folder):
        for file in files:
                if '.txt' not in file: continue
                path = os.path.join(folder,file)
                f = open(path,'r')
                text = f.readlines()[0]
                if '_' in file: 
                        name = file.split('_')[0]
                        if name not in searchList: continue
                        if name not in underscores.keys(): 
                                underscores[name] = text
                        else: underscores[name] = underscores[name] + ' ' + text
                else: 
                        name = file.split('.')[0]
                        if name not in searchList: continue
                        fileToText[name] = text
                f.close()
        for name,text in underscores.items():
            fileToText[name] = text
        return fileToText

def collectgrams(keylist, searchword, collectlist):
        for key in keylist:
                if searchword in key.split('_'):
                        collectlist.append(key) 

def getgrams(txt):
    infile = open(txt,'r')
    lines = infile.readlines()
    infile.close()
    allgrams = []
    for line in lines: 
        line = line.split(':')
        ngrams = line[1].strip()
        if '_' in ngrams: 
                ngrams = ngrams.strip().split(' ')
                for n in ngrams:
                        allgrams.append(n)
        else: continue
    return allgrams 

In [ ]:
#change for each time period: index is period - 1
period = open('/srv/data/timeranges.txt', 'r').readlines()[3]
period = period.strip().strip('[').strip(']').replace("'", '').split(', ')

bigramdata = getTexts('/srv/data/relevantEPBodyNOSTOP', period)
bigramtexts = list(bigramdata.values())
bigramnames = list(bigramdata.keys())

In [ ]:
#specify tobacco/drugs searchword list:
searchwords = ['tobacco']
# searchwords = ['drug', 'apothecary', 'confection', 'elixir', 'medicinable', 'medicine', 'arsenic', 'medecine', 'medicament', 'poppy', 'chemic', 'medicinal', 'intoxicate', 'potion', 'mithridate', 'antimony', 'opiate', 'opium']

remainders = [0, 1, 2]
for n in remainders:
    training = []
    testing = []
    for t in bigramtexts:
        #separating by thirds, remainders: 0, 1, 2
        if bigramtexts.index(t) % 3 == n:
            testing.append(t)
        else: 
            words = t.split(' ')
            training.append(words)
    print('section', n+1, '| training:', len(training), '| testing:', len(testing))

    bigrammodel = Phrases(training, min_count=2, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
    trigrammodel = Phrases(bigrammodel[training], min_count=2, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS )
    print('models trained')

    count = 0
    outfile = open('/srv/data/joy/drugtest.txt', 'a+')
    for text in testing:
        #for outputting to txt file, specify here
        name = bigramnames[bigramtexts.index(text)]
        testtext = text.strip().split(' ')

        grams = []
        bgcount = Counter(b for b in bigrammodel[testtext] if len(b.split("_")) > 1)
        tgcount = Counter(t for t in trigrammodel[testtext] if len(t.split("_")) > 2)
        for searchword in searchwords: 
            collectgrams(dict(bgcount).keys(), searchword, grams) 
            collectgrams(dict(tgcount).keys(), searchword, grams)
        
        gramstring = ' '.join(grams)
        outfile.write(name+': '+gramstring+"\n")
        count += 1
        if count%20 == 0:
            print(count, "files processed")
    outfile.close()
print('processing complete')


In [ ]:
p4tobacco = getgrams('/srv/data/periodFeatures/period4features/newtobaccograms.txt')
p1tobacco = getgrams('/srv/data/periodFeatures/period1features/period1tobaccongrams.txt')
p2tobacco = getgrams('/srv/data/periodFeatures/period2features/period2tobaccongrams.txt')
p3tobacco = getgrams('/srv/data/periodFeatures/period3features/period3tobaccongrams.txt')
p5tobacco = getgrams('/srv/data/periodFeatures/period5features/period5tobaccongrams.txt')

In [ ]:
megalist = p4tobacco + p1tobacco + p2tobacco + p3tobacco + p5tobacco
set(megalist)

In [ ]:
p4drugs = getgrams('/srv/data/joy/drugtest.txt')
len(set(p4drugs))

In [ ]:
plant = ['plant', 'hard', 'strong', 'growth', 'sort', 'sophisticate', 'call', 'leaf', 'description', 'dry', 'green', 'head', 'kind', 'manner', 'name', 'part', 'piece', 'property', 'seed', 'sort', 'small', 'clean', 'common', 'petum', 'root', 'quality']
use = ['child', 'sack', 'bruise','box', 'arm', 'grant', 'alymbecke', 'break', 'dress', 'know', 'caste', 'mark', 'cast', 'drawer', 'set', 'leave', 'glass', 'present', 'send', 'rake', 'drop', 'custom', 'forget', 'invention', 'attendance', 'strew','sit', 'whiff', 'cut', 'smoke', 'pipe', 'use', 'take', 'have', 'taker', 'light', 'keep', 'almanac', 'fume', 'help', 'heat', 'humidity', 'humour', 'infusion', 'juice', 'oil', 'paper', 'perfume', 'phlegm', 'poison', 'powder', 'powder', 'cane', 'pudding', 'savour', 'smell', 'starch', 'suffumigation', 'taste', 'apply', 'ash', 'breath', 'consume', 'cure', 'draw', 'cold', 'purge', 'hot', 'make', 'loosen', 'prepare', 'stillify', 'stink', 'stir', 'warm', 'touch', 'unseal', 'vapour', 'water']
region = ['lemnos', 'spanish', 'indian', 'foreign', 'english', 'dutch', 'sovereign', 'sovereignty', 'society', 'england', 'brazilian', 'virginia', 'trinidada']
trade = ['bag', 'seller', 'line', 'sell', 'quantity', 'ounce', 'piece', 'shop', 'alehouse', 'tavern', 'retail', 'house', 'two', 'barter', 'chest', 'commodity', 'employment', 'gift', 'import', 'importation', 'monopoly', 'ounce', 'pound', 'pocket', 'quantity', 'ship', 'store', 'give', 'one', 'bring', 'buy', 'case', 'furnish', 'gain', 'half', 'spend', 'licence', 'trade', 'trial', 'weight']
commodity = ['wine', 'ale', 'drink', 'other', 'weed', 'beer', 'butter', 'chandler', 'corn', 'cucumber', 'cup', 'flax', 'hemlock', 'herb', 'liquorice', 'rice', 'suet', 'sugar', 'sugarcane', 'copper', 'nightshade']
moral = ['reason', 'sleight', 'delectable', 'malady', 'fast', 'bait', 'save', 'entitle', 'prove', 'resist', 'concern', 'well', 'love', 'good', 'much', 'such', 'abuse', 'against', 'banish', 'best', 'commend', 'counterblaste', 'divine', 'drone', 'excellent', 'famous', 'force', 'great', 'lack', 'like', 'little', 'mistake', 'more', 'poor', 'power', 'praise', 'profane', 'require', 'respectless', 'some', 'succour', 'many', 'exceed', 'few', 'precious', 'hurtful', 'ignorant', 'mighty', 'must', 'noble', 'ingrate', 'ordinary', 'ought', 'pity', 'retail', 'right', 'scarce', 'sin', 'best', 'wholesome', 'immoderate', 'virtue', 'vow', 'hurt', 'worse' ,'yes', 'trick', 'unwholesome', 'want']
people = ['your', 'thy', 'man', 'they', 'doctor', 'our', 'his', 'their', 'main', 'chief', 'farmer', 'gentleman', 'lady', 'maker', 'own', 'she', 'thou', 'bartholomew', 'boy', 'charles', 'woman', 'you']
misc = ['term', 'tob', 'fac', 'dru', 'ser', 'nei', 'maci', 'lemnos', 'after', 'when', 'become', 'almost', 'often', 'upon', 'there', 'all', 'still', 'out', 'may', 'any', 'than', 'whereas', 'this', 'nor', 'itself', 'which', 'that', 'not', 'will', 'into', 'but', 'within', 'shall', 'each', 'but', 'only', 'come', 'about', 'again', 'also', 'among', 'before', 'bring', 'cause', 'even', 'here', 'how', 'if', 'let', 'return', 'then', 'therefore', 'therewith', 'these', 'this', 'although', 'among', 'afterward', 'forthwith', 'thereby', 'yet', 'because', 'bind', 'both', 'can', 'cause', 'come', 'either', 'engender', 'especial', 'etc', 'ever', 'fill', 'find', 'first', 'fit', 'former', 'forsooth', 'from', 'heretofore', 'likewise', 'may', 'mean', 'neither', 'nothing', 'now', 'those', 'what', 'once', 'only', 'out', 'regard', 'ready', 'say', 'see', 'seem', 'shall', 'sure', 'same', 'though', 'through', 'thing', 'till', 'too', 'unless', 'upon', 'think', 'upon', 'very', 'whatsoever', 'when', 'where', 'wherein', 'while', 'who', 'within', 'utter', 'wherewith', 'whether', 'why']

In [ ]:
print('plant:', sorted(set(plant)))
print('use:', sorted(set(use)))
print('region:', sorted(set(region)))
print('trade:', sorted(set(trade)))
print('commodity:', sorted(set(commodity)))
print('people:', sorted(set(people)))
print('moral:', sorted(set(moral)))
print('misc:', sorted(set(misc)))


In [ ]:
plantgrams = []
usegrams = []
regiongrams = []
tradegrams = []
commoditygrams = []
moralgrams = []
peoplegrams = []
miscgrams = []
leftover = []

for ngram in set(megalist):
    for word in ngram.split('_'):
        if word in people:
            peoplegrams.append(ngram)
            break
        elif word in plant:
            plantgrams.append(ngram)
            break
        elif word in use:
            usegrams.append(ngram)
            break
        elif word in region:
            regiongrams.append(ngram)
            break
        elif word in trade:
            tradegrams.append(ngram)
            break
        elif word in commodity:
            commoditygrams.append(ngram)
            break 
        elif word in moral:
            moralgrams.append(ngram)
            break
        elif word in misc:
            miscgrams.append(ngram)
            break
        else: 
            continue

# print('plant:', plantgrams)
# print('use:', usegrams)
# print('region:', regiongrams)
# print('trade:', tradegrams)
# print('commodity:', commoditygrams)
# print('moral:', moralgrams)
# print('people:', peoplegrams)
# print('function:', functiongrams)
# print('leftover:', leftover)

yuck = ['tobacco_tobacco']
categorised = plantgrams + usegrams + regiongrams + tradegrams + commoditygrams + moralgrams + peoplegrams + miscgrams
leftover = set(megalist) - set(categorised) - set(yuck)
leftover

In [ ]:
dictoutfile = open('/srv/data/joy/ngramdict.txt', 'a+')
dictoutfile.write('Plant: '+', '.join(set(plantgrams))+"\n")
dictoutfile.write('People: '+', '.join(set(peoplegrams))+"\n")
dictoutfile.write('Usage: '+', '.join(set(usegrams))+"\n")
dictoutfile.write('Regions: '+', '.join(set(regiongrams))+"\n")
dictoutfile.write('Trade: '+', '.join(set(tradegrams))+"\n")
dictoutfile.write('Commodity: '+', '.join(set(commoditygrams))+"\n")
dictoutfile.write('Ethics: '+', '.join(set(moralgrams))+"\n")
dictoutfile.write('Miscellaneous: '+', '.join(set(miscgrams))+"\n")

dictoutfile.close()